<a href="https://colab.research.google.com/github/eoguzakin/Robustness-of-AI-Generated-Image-Detection-Against-Localized-Inpainting-Attacks/blob/main/Preliminary_UniversalFakeDetect_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################################################
#
#           FINAL, SCALABLE SCRIPT FOR PRELIMINARY EXPERIMENT
#
#   This script is a self-contained, robust pipeline to:
#   1. Set the number of images to test.
#   2. Set up a clean, unified Hugging Face environment.
#   3. Download the specified number of REAL images.
#   4. Use a 'diffusers' pipeline to create FAKE inpainted images.
#   5. Use CLIP (UniversalFakeDetect logic) to get a final accuracy score.
#
######################################################################

# --- Step 1: Configuration ---
NUM_IMAGES_TO_TEST = 200  # <<< You can change this number (e.g., to 200 or 500)

# --- Step 2: Install a Clean, Unified Environment ---
print(f"--- [1/5] Installing a unified Hugging Face environment for {NUM_IMAGES_TO_TEST} images... ---")
!pip install diffusers transformers accelerate torch datasets Pillow ftfy regex tqdm -q
!pip install git+https://github.com/openai/CLIP.git -q
print("--- Environment setup complete. ---")

# --- Step 3: Download REAL Images ---
print(f"\n--- [2/5] Downloading {NUM_IMAGES_TO_TEST} real images... ---")
from datasets import load_dataset
import os

real_dir = "/content/data/real"
os.makedirs(real_dir, exist_ok=True)

try:
    # Use the 'beans' dataset - it's simple and reliable
    dataset = load_dataset("beans", split=f"train[:{NUM_IMAGES_TO_TEST}]", trust_remote_code=True)
    for i, example in enumerate(dataset):
        image = example['image'].convert("RGB").resize((512, 512))
        image.save(os.path.join(real_dir, f"{i:05d}.png"))
    print(f"--- Successfully downloaded and saved {NUM_IMAGES_TO_TEST} real images. ---")
except Exception as e:
    print(f"--- ERROR downloading real images: {e} ---")

# --- Step 4: Create FAKE Images with Stable Diffusion Inpainting ---
print(f"\n--- [3/5] Creating {NUM_IMAGES_TO_TEST} fake (inpainted) images... ---")
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from tqdm import tqdm
import numpy as np

fake_dir = "/content/data/fake"
os.makedirs(fake_dir, exist_ok=True)

try:
    # Load the inpainting pipeline
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
        torch_dtype=torch.float16,
        safety_checker=None
    ).to("cuda")

    real_images = os.listdir(real_dir)
    for filename in tqdm(real_images, desc="Inpainting Images"):
        init_image = Image.open(os.path.join(real_dir, filename))

        # Create a random mask with the correct data type
        mask_array = (np.random.rand(512, 512) > 0.7) * 255
        mask_image = Image.fromarray(mask_array.astype(np.uint8))

        prompt = "photorealistic, high detail"
        image = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
        image.save(os.path.join(fake_dir, filename))

    print(f"--- Successfully created {NUM_IMAGES_TO_TEST} fake images. ---")
except Exception as e:
    print(f"--- ERROR during inpainting: {e} ---")


# --- Step 5: Run the UniversalFakeDetect (CLIP) Evaluation ---
print(f"\n--- [4/5] Running CLIP-based evaluation on {NUM_IMAGES_TO_TEST*2} total images... ---")
import clip

try:
    device = "cuda"
    model, preprocess = clip.load("ViT-L/14", device=device)

    def get_features(directory):
        features = []
        image_files = sorted(os.listdir(directory))
        for filename in tqdm(image_files, desc=f"Encoding {os.path.basename(directory)}"):
            img_path = os.path.join(directory, filename)
            image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                features.append(model.encode_image(image))
        return torch.cat(features)

    real_features = get_features(real_dir)
    fake_features = get_features(fake_dir)

    # Calculate the mean feature vector (centroid) for each class
    real_centroid = real_features.mean(dim=0)
    fake_centroid = fake_features.mean(dim=0)

    # Normalize the centroids to be unit vectors
    real_centroid /= torch.linalg.norm(real_centroid)
    fake_centroid /= torch.linalg.norm(fake_centroid)

    print("\n--- [5/5] Calculating preliminary results... ---")
    correct_predictions = 0

    # Test real images: they should be closer to the real centroid
    real_sim_real = real_features @ real_centroid
    real_sim_fake = real_features @ fake_centroid
    correct_predictions += (real_sim_real > real_sim_fake).sum().item()

    # Test fake images: they should be closer to the fake centroid
    fake_sim_real = fake_features @ real_centroid
    fake_sim_fake = fake_features @ fake_centroid
    correct_predictions += (fake_sim_fake > fake_sim_real).sum().item()

    total_images = len(real_features) + len(fake_features)
    accuracy = (correct_predictions / total_images) * 100

    print("\n\n" + "="*30)
    print("--- PRELIMINARY RESULTS ---")
    print("="*30)
    print(f"Detector Evaluated: UniversalFakeDetect (CLIP)")
    print("-" * 30)
    print(f"Total Images Evaluated: {total_images}")
    print(f"Correct Predictions:    {correct_predictions}")
    print("-" * 30)
    print(f"PRELIMINARY ACCURACY: {accuracy:.2f}%")
    print("="*30)

except Exception as e:
    print(f"\n--- An error occurred during the evaluation: {e} ---")

--- [1/5] Installing a unified Hugging Face environment for 200 images... ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'beans' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'beans' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of 

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Inpainting Images: 100%|██████████| 200/200 [26:57<00:00,  8.09s/it]


--- Successfully created 200 fake images. ---

--- [4/5] Running CLIP-based evaluation on 400 total images... ---


100%|███████████████████████████████████████| 890M/890M [00:41<00:00, 22.7MiB/s]
Encoding fake: 100%|██████████| 200/200 [00:06<00:00, 30.49it/s]



--- [5/5] Calculating preliminary results... ---


--- PRELIMINARY RESULTS ---
Detector Evaluated: UniversalFakeDetect (CLIP)
------------------------------
Total Images Evaluated: 400
Correct Predictions:    387
------------------------------
PRELIMINARY ACCURACY: 96.75%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
